# Load Packages

In [1]:
%matplotlib inline 
from ifis_tools import database_tools as db 
from ifis_tools import asynch_manager as am 
from ifis_tools import auxiliar as aux
from wmf import wmf
import pandas as pd 
import numpy as np 
import os 
import pylab as pl
from string import Template
from param_ident import core 

No se logra importar deap tools, por lo tanto se deshabilita SimuBasin.Calib_NSGAII


In [82]:
names = {'south': {'path': 'data/for_hlm/south_skunk/'},
        'turkey':{'path':'data/for_hlm/turkey/'}}

# Define Watershed setups

In this step we will produce the setups of the watersheds making variations of $v_0$ and $\lambda_1$ for different sizes of sub-watersheds.

In [10]:
#Get the watershed
south = am.hlmModel(linkid=272678, ExtraParams=['h_order'])

#Temporal rvr to get the topology of the watershed
south.write_rvr('data/for_hlm/south_skunk/south_skunk.rvr')
topo = south.topo.copy()
topo.set_index('link_id', inplace = True)
idx = south.Table.index.intersection(topo.index)
south.Table['dest'] = topo['id'].loc[idx]

In [11]:
#Get the watershed
turkey = am.hlmModel(linkid=434514, ExtraParams=['h_order'])

#Temporal rvr to get the topology of the watershed
turkey.write_rvr('data/for_hlm/turkey/turkey.rvr')
topo = turkey.topo.copy()
topo.set_index('link_id', inplace = True)
idx = turkey.Table.index.intersection(topo.index)
turkey.Table['dest'] = topo['id'].loc[idx]

In [12]:
wat = {'south': south, 'turkey': turkey}

## Obtain sub-watersheds at different prun-levels

Here we include levels from 7 to 4, then we will produce random values of $v_0$ and $\lambda_1$ using a cascade approach.

In [ ]:
# obtain the sub-watersheds for each one (this takes time try not to run it, instead read the data)
core.Get_sub_watersheds(wat['south'], 7, 3)
core.Get_sub_watersheds(wat['turkey'], 7, 3)
# Save to csv files to avoid doing this again.
wat['south'].Table.to_csv('data/for_hlm/south_skunk/watershed_table.csv')
wat['turkey'].Table.to_csv('data/for_hlm/turkey/watershed_table.csv')

In [74]:
wat['south'].Table = pd.read_csv('data/for_hlm/south_skunk/watershed_table.csv', index_col=0)
wat['turkey'].Table = pd.read_csv('data/for_hlm/turkey/watershed_table.csv', index_col=0)

## Distribute $v_0$ and $\lambda_1$ using the cascade

Produce $v_0$ values for each level 

In [77]:
for name, seed in zip(['turkey','south'], [[0.15,0.2,0.18,0.25,0.3,0.35,0.4], [0.15,0.3,0.22]]):
    for level in [6, 5, 4]:
        core.cascade_values(wat[name].Table, 'vo_cas', level, seed)
        wat[name].Table.rename(columns = {'vo_cas': 'vo'+str(level)}, inplace = True)

Produce $\lambda_1$ values for each level 

In [80]:
for name, seed in zip(['turkey','south'], [[0.19,0.22,0.2,0.22,0.2,0.23,0.3], [0.18,0.23,0.25]]):
    for level in [6, 5, 4]:
        core.cascade_values(wat[name].Table, 'vo_cas', level, seed)
        wat[name].Table.rename(columns = {'vo_cas': 'la'+str(level)}, inplace = True)

Save the results to a couple of tables with the distributed parameters.

In [84]:
for name in names.keys():
    wat[name].Table.to_csv(names[name]['path']+'water_virtual.csv')